In [1]:
import pickle
import numpy as np
from random import gauss

import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, LSTM, concatenate, Multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import Sequence

import os
os.makedirs('trained_model_baseline', exist_ok=True)

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
def load_datasets(path='simulation_data/', string='M1_M20'):
    if not path.endswith('/'):
        path += '/'
    filenames = ['x1_train', 'x2_train', 'y_train', 'x1_val', 'x2_val', 'y_val', 'x1_test', 'x2_test', 'y_test']
    data = []
    for file_name in filenames:
        file_path = path + file_name + '_' + string + '.pkl'
        with open(file_path, 'rb') as f:
            data.append(pickle.load(f))
    return data

x1_train, x2_train, y_train, x1_val, x2_val, y_val, x1_test, x2_test, y_test = \
    load_datasets(path='simulation_data/', string='M1_M20')

print("x1_train shape:", x1_train.shape)
print("x2_train shape:", x2_train.shape)
print("y_train shape:", y_train.shape)
print("x1_val shape:", x1_val.shape)
print("x2_val shape:", x2_val.shape)
print("y_val shape:", y_val.shape)
print("x1_test shape:", x1_test.shape)
print("x2_test shape:", x2_test.shape)
print("y_test shape:", y_test.shape)

x1_train shape: (4950000, 4)
x2_train shape: (4950000, 21, 12)
y_train shape: (4950000, 20)
x1_val shape: (50000, 4)
x2_val shape: (50000, 21, 12)
y_val shape: (50000, 20)
x1_test shape: (100000, 4)
x2_test shape: (100000, 7, 12)
y_test shape: (100000, 20)


In [3]:
def check_one_hot(arr):
    if arr.ndim != 2 or arr.shape[1] != 20:
        return False
    row_sums = np.sum(arr, axis=1)
    return np.all((row_sums == 1) & (np.count_nonzero(arr, axis=1) == 1))

def print_label_info(arr, name, n_show=5, seed=42):
    print(f"{name} shape: {arr.shape}")
    if check_one_hot(arr):
        print("Format: one-hot encoding\n")
        unique_indices = np.unique(np.argmax(arr, axis=1))
        print(f"Unique mechanism labels in {name}: {[f'M{idx+1:02d}' for idx in unique_indices]}")
        np.random.seed(seed)
        indices = np.random.choice(arr.shape[0], n_show, replace=False)
        for i, idx in enumerate(indices):
            label = np.argmax(arr[idx])
            one_hot_str = ', '.join(str(int(x)) for x in arr[idx])
            print(f"Sample {i+1} (index {idx}): One-hot: [{one_hot_str}], Mechanism Label: M{label+1:02d}")
    else:
        print("Format: NOT one-hot encoding")
    print()
    print('-'*80)

print_label_info(y_train, "y_train")
print_label_info(y_val, "y_val")
print_label_info(y_test, "y_test")

print("The label for each sample is the corresponding mechanism used to generate its kinetic data, encoded as a one-hot vector as follows:\n")
for i in range(20):
    vec = ['0']*20
    vec[i] = '1'
    print(f"M{i+1:>2}: ({', '.join(vec)})")

y_train shape: (4950000, 20)
Format: one-hot encoding

Unique mechanism labels in y_train: ['M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20']
Sample 1 (index 2647414): One-hot: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Mechanism Label: M03
Sample 2 (index 1027977): One-hot: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Mechanism Label: M06
Sample 3 (index 3993827): One-hot: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Mechanism Label: M07
Sample 4 (index 2955156): One-hot: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Mechanism Label: M07
Sample 5 (index 319154): One-hot: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Mechanism Label: M01

--------------------------------------------------------------------------------
y_val shape: (50000, 20)
Format: one-hot encoding

Unique mechanism labels in y_val: ['M01', 'M02

### Baseline Model

In [ ]:
def build_lstm_classifier_baseline(input1_shape, input2_shape, output_shape):
    initial_concentrations = Input(shape=input1_shape)
    kinetics = Input(shape=(None, input2_shape[-1]))
    
    c = Dense(64, activation='relu')(initial_concentrations)
    
    k = LSTM(64, return_sequences=True)(kinetics)
    k = LSTM(64, return_sequences=False)(k)
    
    pred_c = Dense(output_shape[1], activation='softmax', name='static_output')(c)
    pred_k = Dense(output_shape[1], activation='softmax', name='dynamic_output')(k)
    
    avg_pred = tf.keras.layers.Average()([pred_c, pred_k])
    
    model = Model(inputs=[initial_concentrations, kinetics], outputs=avg_pred)
    return model

class KineticsBatchGenerator(Sequence):
    def __init__(self, X, y, tps, error_range, seed_value=1, batch_size=1024, shuffle=True):
        self.x1, self.x2 = X
        self.y = y
        self.tps = tps
        self.error_range = error_range
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.total_batches = self.__len__()
        self.n_runs = 4
        self.species = [1, 2]
        self.error_species = len(self.species)
        self.error_dict = {key: [] for key in error_range}
        np.random.seed(seed_value)
        self.randomstate = np.random.default_rng(seed_value)
        self.on_epoch_end()
        examples, timepoints, curves = self.x2.shape
        columns = curves // self.n_runs
        self.index_species = []
        for i in range(self.n_runs):
            t_species = [a + (i * columns) for a in self.species]
            self.index_species = self.index_species + t_species

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def __getitem__(self, index):
        return self.__data_generation(index)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.y))
        if self.shuffle:
            np.random.shuffle(self.indexes)
        for error in self.error_range:
            if error != 0:
                preu = np.array([gauss(0, error / 100) for _ in range(self.batch_size * np.max(self.tps) * self.error_species * self.n_runs)])
                self.error_dict[error] = np.reshape(preu, (self.batch_size, np.max(self.tps), self.error_species * self.n_runs))

    def __data_generation(self, index):
        x1 = self.x1[self.batch_size * index: self.batch_size * (index + 1)]
        x2 = self.introduce_error(self.trim_x2(self.x2[self.batch_size * index: self.batch_size * (index + 1)], self.tps, index))
        y = self.y[self.batch_size * index: self.batch_size * (index + 1)]
        x = [x1, x2]
        return x, y

    def trim_x2(self, original_x2, tps, index):
        original_tps = original_x2.shape[1]
        n_tps = self.randomstate.choice(tps)
        idx = np.sort(np.append(self.randomstate.choice(original_tps - 1, n_tps, replace=False) + 1, [0]))
        return original_x2[:, idx].copy()

    def introduce_error(self, x2):
        examples, timepoints, curves = x2.shape
        error = self.randomstate.choice(self.error_range)
        if error != 0:
            x2[:, 1:, self.index_species] = x2[:, 1:, self.index_species] + self.error_dict[error][:, 0:timepoints - 1]
        return x2

def train_mechanism_model_baseline(
    tps = [3,4,5,6,7,8,9,10,15,20],
    error_range = [0, 0.5, 1.2],
    epochs = 1000,
    start = 0,
    batch_size_training = 2048
):
    x1_train, x2_train, y_train, x1_val, x2_val, y_val, x1_test, x2_test, y_test = load_datasets()

    model = build_lstm_classifier_baseline(
        input1_shape = x1_train.shape[1:],
        input2_shape = x2_train.shape[1:],
        output_shape = y_train.shape
    )
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    early_stopping = EarlyStopping(
        monitor='val_categorical_accuracy',
        verbose=1,
        patience=100,
        mode='max',
        restore_best_weights=True
    )
    checkpoint = ModelCheckpoint(
        "trained_model_baseline/best_model_weights_baseline",
        monitor='val_categorical_accuracy',
        verbose=1,
        save_best_only=True,
        save_weights_only=True
    )

    history = model.fit(
        KineticsBatchGenerator([x1_train, x2_train], y_train, tps, error_range, batch_size=batch_size_training),
        validation_data=KineticsBatchGenerator([x1_val, x2_val], y_val, tps, error_range, batch_size=25),
        callbacks=[checkpoint, early_stopping],
        epochs=start+epochs,
        initial_epoch=start
    )

    with open("trained_model_baseline/training_history_baseline.pkl", "wb") as f:
        pickle.dump(history.history, f)

    model.save('trained_model_baseline/M1_20_model_classification_baseline')
    
    print('Training completed. Optimal weights saved to: trained_model_baseline/best_model_weights_baseline')
    print('Training history saved to: trained_model_baseline/training_history_baseline.pkl')
    print('Trained model saved to: trained_model_baseline/M1_20_model_classification_baseline')

    return model

model = train_mechanism_model_baseline()

Train for 2416 steps, validate for 2000 steps
Epoch 1/1000
2415/2416 [============================>.] - ETA: 0s - loss: 2.8412 - categorical_accuracy: 0.1157
Epoch 00001: val_categorical_accuracy improved from -inf to 0.13546, saving model to trained_model_baseline/best_model_weights_baseline
2416/2416 [==============================] - 26s 11ms/step - loss: 2.8412 - categorical_accuracy: 0.1157 - val_loss: 2.7685 - val_categorical_accuracy: 0.1355
Epoch 2/1000
2412/2416 [============================>.] - ETA: 0s - loss: 2.7344 - categorical_accuracy: 0.1495
Epoch 00002: val_categorical_accuracy improved from 0.13546 to 0.16052, saving model to trained_model_baseline/best_model_weights_baseline
2416/2416 [==============================] - 24s 10ms/step - loss: 2.7344 - categorical_accuracy: 0.1495 - val_loss: 2.6948 - val_categorical_accuracy: 0.1605
Epoch 3/1000
2413/2416 [============================>.] - ETA: 0s - loss: 2.6743 - categorical_accuracy: 0.1702
Epoch 00003: val_categori